In [68]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [69]:
from medvqa.models.huggingface_utils import (
    CachedTextEmbeddingExtractor,
    SupportedHuggingfaceMedicalBERTModels,
)
from medvqa.utils.files import (
    load_jsonl, load_pickle, load_class_specific_regex,
)
from medvqa.utils.math import rank_vectors_by_dot_product, rank_vectors_by_average_dot_product
import numpy as np

In [70]:
from importlib import reload
import medvqa

In [498]:
reload(medvqa.utils.files)
reload(medvqa.utils.common)

<module 'medvqa.utils.common' from '/home/pamessina/medvqa/medvqa/utils/common.py'>

In [7]:
integrated_fact_metadata_filepath = "/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"

In [8]:
from medvqa.utils.constants import MULTIDATASET_UNIFIED_CLASSES
phrases = MULTIDATASET_UNIFIED_CLASSES
print(len(phrases))

297


In [55]:
# Create embedding extractor
embedding_extractor = CachedTextEmbeddingExtractor(
    model_name='microsoft/BiomedVLP-CXR-BERT-specialized',
    model_checkpoint_folder_path='/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)',
    device='cuda',
)

checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']


In [9]:
# Load integrated fact metadata
print('Loading integrated fact metadata from:', integrated_fact_metadata_filepath)
integrated_fact_metadata = load_jsonl(integrated_fact_metadata_filepath)

Loading integrated fact metadata from: /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl


In [295]:
# Extract facts
all_facts = set()
for x in integrated_fact_metadata:
    fact = x['fact']
    if fact: all_facts.add(fact)
all_facts = list(all_facts)
all_facts.sort()
print(f'Loaded {len(all_facts)} facts.')

Loaded 591920 facts.


In [408]:
for f in facts:
    if 'pneumothorax' in f:
        print(f)
        break

pleural surfaces without pneumothorax


In [296]:
fact_embeddings = embedding_extractor.compute_text_embeddings(all_facts)

100%|███████████████████████████████████████████████████████████████████████| 591920/591920 [00:01<00:00, 445021.78it/s]


In [59]:
phrase_embeddings = embedding_extractor.compute_text_embeddings(phrases)

100%|█████████████████████████████████████████████████████████████████████████████| 297/297 [00:00<00:00, 106198.49it/s]


In [11]:
i = phrases.index('bullae')
i

38

In [12]:
idxs = rank_vectors_by_dot_product(fact_embeddings, phrase_embeddings[i])

In [14]:
for k in range(100):
    print(all_facts[idxs[k]], np.dot(fact_embeddings[idxs[k]], phrase_embeddings[i]))

bullae 0.99999994
bulla 0.9935278
bulla formation 0.990708
blebs 0.9893153
bleb 0.988828
bullous changes 0.98770356
bullous change 0.9871189
cysts 0.9870987
cystic changes 0.97311807
cystic abnormalities 0.96693194
paraseptal bullae 0.9661053
blebectomy 0.9629918
cystic lucencies 0.96020746
unusual bulla 0.95713097
pneumatocele 0.9561011
areas of bullous change 0.9550576
sebaceous cyst 0.9536319
underlying bleb 0.95282066
multiple bullae 0.9515826
bullous disease 0.95083356
multiple bulla 0.9505608
pneumatocele formation 0.94924504
PROMINENT bullous change 0.94900554
bipical blebs 0.94535685
bullae in the lungs 0.9449865
prominent bullous formation in the region 0.9439609
pneumatoceles 0.9427241
multiple blebs 0.9425837
seed cysts 0.93939626
known bleb in the region 0.93627673
bullae filled with fluid 0.93498886
bulla within the lungs 0.93294966
bulla formation in the lungs 0.9322265
evidence of bullous disease 0.93140024
bullas in the lungs 0.93029946
several large bullae 0.9298261
bu

In [611]:
# query = 'cavity'
# query_embedding = embedding_extractor.compute_text_embeddings([query], update_cache_on_disk=False)[0]
# queries = [
#     'cavity',
#     'lung cavity',
#     'cavitating lesion',
#     'cavitation',
# ]
queries = [
    'enlarged PA',
#     'batwing shadowing',
#     'airspace shadowing',
#     'alveolar shadowing',
]
query_embeddings = embedding_extractor.compute_text_embeddings(queries, update_cache_on_disk=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14266.34it/s]


Computing embeddings for 1 new texts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading model weights from /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt
Examples of keys in loaded state dict but not in model:
  nli_classifier.bias
  fact_decoder_input_layer.bias
  health_status_classifier.weight
  fact_decoder.decoder.layers.0.norm1.bias
  category_classifier.weight
  chest_imagenome_obs_classifier.weight
  fact_decoder_input_layer.weight
  spert_size_embeddings.weight
  fact_decoder.decoder.layers.0.multihead_attn.out_proj.bias
  fact_decoder.decoder.layers.0.self_attn.out_proj.weight


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15363.75it/s]


In [612]:
# idxs = rank_vectors_by_dot_product(fact_embeddings, query_embedding)
idxs = rank_vectors_by_average_dot_product(fact_embeddings, query_embeddings)

In [581]:
from tqdm import tqdm
regex = load_class_specific_regex('VinDr-CXR', "Tuberculosis")
filtered = [f for f in tqdm(all_facts, mininterval=2) if regex.search(f)]
len(filtered)

100%|███████████████████████████████████████████████████████████████████████| 591920/591920 [00:03<00:00, 159476.66it/s]


533

In [585]:
import random
random.sample(filtered, 50)

['no TB reactive a shin',
 'no radiographic evidence to suggest tuberculosis',
 'no non-active TB',
 'linear scarring pattern potentially reflecting old tuberculous disease',
 'possible reactivation tuberculosis in the right upper lobe',
 'no radiographic evidence for active pulmonary TB',
 'no findings to suggest latent pulmonary tuberculosis infection',
 'no radiographic evidence of latent pulmonary tuberculosis infection',
 'likely old tuberculous disease in bilateral apices',
 'no evidence of inactive TB',
 'no known recent TB',
 'TB is less likely',
 'no evidence of old tuberculous disease',
 'residual of tuberculosis in the right lung apex',
 'no evidence of tuberculous infection',
 'no radiographic evidence of latent tuberculosis',
 'nodules related to prior TB',
 'no signs of non-active tuberculosis',
 'reticular changes consistent with known provided history of active tuberculosis',
 'findings suggestive of old tuberculosis infection',
 'calcifications consistent with history 

In [613]:
# for k in range(100):
#     print(f'{np.dot(fact_embeddings[idxs[k]], query_embedding):.4f} {all_facts[idxs[k]]}')
for k in range(50):
#     k = -k-1
    print(f'{np.dot(fact_embeddings[idxs[k]], query_embeddings.T).mean():.4f} {all_facts[idxs[k]]}')

0.9308 enlargement of the main PA
0.9117 substantially increased size
0.9006 significantly increased size
0.9001 dense for their size
0.8989 greater size
0.8986 expansion in the AP dimension
0.8936 increased size of the abnormality
0.8908 probably larger size
0.8880 widening in the prior exam
0.8852 enlargement compared to prior study
0.8833 increased size from the prior CT
0.8797 accentuated enlargement by the projection
0.8787 changes in size
0.8770 exaggerated size due to AP projection
0.8769 increase in size
0.8756 increased AP dimension
0.8750 perceived enlargement
0.8728 substantially enlarged
0.8678 increased size
0.8671 increased size compared to the prior study
0.8642 prominent size
0.8602 larger appearance
0.8599 multichamber enlargement
0.8550 contour appears enlarged
0.8536 exaggerated enlargement due to the AP projection
0.8536 enlarged size
0.8530 large
0.8487 increase in size compared to prior exam
0.8480 large AP distance
0.8464 increased size of the silhouette
0.8436 e

In [340]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(606074,61621903).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.65

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.65
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(606074,61621903).jsonl...
Loaded 505377 facts (skipped 100693 facts)
len(facts) before removing invalid facts: 505377
100%|█████████████████████████████████| 505377/505377 [01:05<00:00, 7694.00it/s]
len(facts) after removing invalid facts: 503311
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/503311 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 702198
self.cache["embeddings"].shape = (702198, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 208521.64it/s]
fact_embedd

In [136]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"\
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.5

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.5
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 876541
self.cache["embeddings"].shape = (876541, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 157604.79it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (250466, 128)
anchor_fact_e

In [2]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"\
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.6

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.6
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 252372 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 855787
self.cache["embeddings"].shape = (855787, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 260881.32it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (252372, 128)
anchor_fact_embeddings.shape: (297, 128

In [33]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.7

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.7
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 252372 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 855787
self.cache["embeddings"].shape = (855787, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 264089.10it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (252372, 128)
anchor_fact_embeddings.shape: (297, 128

In [149]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl"

-rw-rw-r-- 1 pamessina socvis 42M Jan  7 16:41 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl'


In [64]:
tmp = load_pickle("/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl")

In [65]:
tmp.keys()

dict_keys(['anchor_facts', 'preserved_general_facts', 'preserved_facts', 'most_similar_to_fact', 'most_similar_to_general_fact'])

In [67]:
len(tmp['preserved_facts']), len(tmp['preserved_general_facts'])

(416649, 207909)

In [75]:
len(set(tmp['preserved_facts']) | set(tmp['preserved_general_facts']))

550410

In [142]:
tmp['preserved_facts'].index('pneumothorax')

265832

In [143]:
class_idx = tmp['anchor_facts'].index('reticulation')
class_idx

252

In [144]:
idxs1 = [i for i, x in enumerate(tmp['most_similar_to_general_fact']) if x is not None and x[1] == class_idx]
idxs1.sort(key=lambda idx: tmp['most_similar_to_general_fact'][idx][0], reverse=True)
idxs2 = [i for i, x in enumerate(tmp['most_similar_to_fact']) if x is not None and x[1] == class_idx]
idxs2.sort(key=lambda idx: tmp['most_similar_to_fact'][idx][0], reverse=True)
len(idxs1), len(idxs2)

(42, 33)

In [145]:
for i in range(0, min(200, len(idxs1))):
    i = idxs1[i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_general_fact'][i][1]]
    fac = tmp['preserved_general_facts'][i]
    sim = tmp['most_similar_to_general_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9706587791442871	reticular prominence (reticulation)
0.970277726650238	reticulation seen (reticulation)
0.9595723748207092	background reticulation (reticulation)
0.958702564239502	reticulation pattern observed (reticulation)
0.9443132877349854	conspicuity of reticular architecture (reticulation)
0.9388192892074585	coarse reticulation (reticulation)
0.9381308555603027	prominent reticular pattern (reticulation)
0.9291142225265503	coarse reticulations (reticulation)
0.9289706945419312	interstitial abnormality with B-lines (reticulation)
0.9217033386230469	interstitial abnormality including B-lines (reticulation)
0.9206329584121704	cortical reticulation (reticulation)
0.9119927883148193	reticulation most pronounced (reticulation)
0.9075450301170349	generalized reticulation (reticulation)
0.9017454981803894	dense streak (reticulation)
0.8959839344024658	coarse markings (reticulation)
0.8754942417144775	new reticular pattern (reticulation)
0.8647550344467163	coarsened markings (reticulatio

In [146]:
for i in range(0, min(200, len(idxs2))):
    i = idxs2[i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_fact'][i][1]]
    fac = tmp['preserved_facts'][i]
    sim = tmp['most_similar_to_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9797871708869934	reticulations (reticulation)
0.9560151100158691	superimposed reticulation (reticulation)
0.9431335926055908	reticular prominence process (reticulation)
0.9428203701972961	conspicuity of the reticular architecture (reticulation)
0.9388192892074585	coarse reticulation (reticulation)
0.9289706945419312	interstitial abnormality with B-lines (reticulation)
0.9217033386230469	interstitial abnormality including B-lines (reticulation)
0.8746028542518616	interstitial abnormality best seen on the lateral view (reticulation)
0.8647550344467163	coarsened markings (reticulation)
0.8438680171966553	prominent tissue markings (reticulation)
0.838477373123169	intraparenchymal abnormalities (reticulation)
0.8379271030426025	calcified parenchymal changes (reticulation)
0.8340967893600464	intraparenchymal abnormality (reticulation)
0.804874062538147	B-lines compared with the prior radiograph (reticulation)
0.7993770241737366	diffuse interstitial abnormality in the lateral view (reticula

In [443]:
for x in integrated_fact_metadata:
    if x['fact'] == "right lower lobe consolidation":
        print(x)
        break

{'fact': 'right lower lobe consolidation', 'metadata': {'anatomical location': 'right lower lobe', 'detailed observation': 'consolidation', 'short observation': 'consolidation', 'category': 'anatomical finding', 'health status': 'abnormal', 'prev_study_comparison?': 'no', 'comparison status': ''}, 'extraction_method': 'gpt-3.5-turbo-0613'}


In [150]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli" \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--facts_similar_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--mlp_batch_size 1000 \
--mlp_num_workers 4 \
--mlp_nli_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)" \
--mlp_nli_entailment_threshold 0.1723824292421341 \
--num_clusters 300 \
--max_negative_facts_per_report 500

Running _find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli()...
Reading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl...
n_reports: 227835
len(all_facts): 586352
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(all_facts): 769532
Loading /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl...
len(preserved_facts): 416649
len(preserved_general_facts): 207909
len(all_facts): 769532
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/769532 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/

In [15]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl"

-rw-rw-r-- 1 pamessina socvis 908M Jan  7 17:31 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl'


In [16]:
aux = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl')

In [17]:
aux.keys()

dict_keys(['facts', 'embeddings', 'dicom_id_to_pos_neg_facts'])

In [18]:
len(aux['facts'])

769532

In [19]:
aux['embeddings'].shape

(769532, 128)

In [20]:
dicom_id_to_pos_neg_facts = aux['dicom_id_to_pos_neg_facts']

In [21]:
dicom_ids = list(dicom_id_to_pos_neg_facts.keys())

In [7]:
integrated_report_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl')

In [8]:
from medvqa.datasets.mimiccxr import get_detailed_metadata_for_dicom_id
from medvqa.utils.files import read_txt
from medvqa.utils.logging import print_bold

In [172]:
integrated_report_facts[metadata['report_index']]

{'report_idx': 3051,
 'background': 'indication: year old man s/p R thoracotomy w/ multiple wedge resections // R/O PTX post CT removal, please do around 2PM R/O PTX post CT removal, please do around 2PM.',
 'findings': 'In comparison with the earlier study of this date, the right chest tube is been removed and there is no evidence of pneumothorax. Otherwise little change in the appearance of the heart and lungs.',
 'impression': '',
 'path': '/mnt/data/mimic-cxr/files/p10/p10405305/s59536698.txt',
 'facts': ['right chest tube has been removed',
  'no evidence of pneumothorax',
  'little change in the appearance of the heart',
  'little change in the appearance of the lungs'],
 'fact_based_report': 'right chest tube has been removed. no evidence of pneumothorax. little change in the appearance of the heart. little change in the appearance of the lungs'}

In [151]:
dicom_id = dicom_ids[5040]
metadata = get_detailed_metadata_for_dicom_id(dicom_id)[0]
print(metadata)
print()
print()
print(read_txt(metadata['filepath']))
print()
print()
facts = integrated_report_facts[metadata['report_index']]['facts']
print('. '.join(facts))
print()
print_bold('All facts:')
facts = integrated_report_facts[metadata['report_index']]['facts']
print(facts)
print()
print_bold('Positive facts:')
positive_facts = [aux['facts'][idx] for idx in dicom_id_to_pos_neg_facts[dicom_id][0]]
print(positive_facts)
print()
print_bold('Excluded positive facts:')
print([f for f in facts if f not in positive_facts])
print()
print_bold('Negative facts:')
for idx in dicom_id_to_pos_neg_facts[dicom_id][1]:
    print(aux['facts'][idx])

{'report_index': 3051, 'part_id': 10, 'subject_id': 10405305, 'study_id': 59536698, 'dicom_id': '32503ac2-0f5e85a1-461b3f33-934b1604-b6ef3d15', 'view_pos': 'LL', 'split': 'train', 'filepath': '/mnt/data/mimic-cxr/files/p10/p10405305/s59536698.txt', 'dicom_id_view_pos_pairs': [('32503ac2-0f5e85a1-461b3f33-934b1604-b6ef3d15', 'LL'), ('ec8ff472-7a5c52c3-1c88fad5-4081d595-9dbe483a', 'AP')]}


                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___ year old man s/p R thoracotomy w/ multiple wedge resections 
 // R/O PTX post CT removal, please do around 2PM      R/O PTX post CT removal,
 please do around 2PM
 
 IMPRESSION: 
 
 In comparison with the earlier study of this date, the right chest tube is
 been removed and there is no evidence of pneumothorax.  Otherwise little
 change in the appearance of the heart and lungs.



right chest tube has been removed. no evidence of pneumothorax. little change in the appearance of the heart. little change 

In [118]:
integrated_fact_metadata[0]

{'fact': 'history of sternal debridement in bilateral pectoralis flaps',
 'metadata': {'reason': 'The statement refers to a medical history involving surgical procedures but does not describe a current observation from a radiology report. It indicates past interventions rather than findings from an imaging study.',
  'too_noisy_or_irrelevant': 'yes',
  'visually_observable': 'no',
  'category': 'does not apply',
  'abnormality_status': 'does not apply',
  'anatomical_location': 'does not apply',
  'general_observation': 'does not apply'},
 'extraction_method': 'gpt-4o-mini'}

In [30]:
from medvqa.scripts.mimiccxr.find_pos_neg_facts_per_report_around_reference_classes import _filter_facts

In [31]:
_unique_facts, _unique_general_facts, _f2gf, banned_facts = _filter_facts(integrated_fact_metadata)

len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts


In [33]:
banned_facts = list(banned_facts)

In [39]:
banned_facts[2]

'no superadded pneumonia'

In [46]:
b_idxs = [i for i,x in enumerate(banned_facts) if 'line' in x.lower()]
len(b_idxs)

1813

In [54]:
query = 'possible artifactual at the right lung base'
# query = banned_facts[b_idxs[10]]
print(query)
print()

for x in integrated_fact_metadata:
    if x['fact'] == query:
        print(x)
        break
print()

q_idx = all_facts.index(query)
sims = np.dot(phrase_embeddings, fact_embeddings[q_idx])
idxs = np.argsort(sims)[::-1]
for idx in idxs:
    print(f'{sims[idx]}  {phrases[idx]}')

possible artifactual at the right lung base

{'fact': 'possible artifactual at the right lung base', 'metadata': {'reason': 'The statement suggests the presence of an artifact in the imaging of the right lung base. Artifacts can affect the interpretation of radiological images and are relevant observations in radiology reports.', 'too_noisy_or_irrelevant': 'no', 'visually_observable': 'yes', 'category': 'technical assessment', 'abnormality_status': 'does not apply', 'anatomical_location': 'right lung base', 'general_observation': 'possible artifactual at the lung base'}, 'extraction_method': 'gpt-4o-mini'}

0.4726397693157196  prominent hila
0.46213287115097046  prominence of hilum
0.4599626064300537  hyperinflation
0.4561922252178192  hilum prominent
0.4299100637435913  hyperaeration
0.42224496603012085  hyperexpanded lungs
0.4140942692756653  lobar/segmental collapse
0.3989369869232178  interstitial opacification
0.3954535722732544  increased interstitial markings
0.3942326605319977 

In [130]:
query_idxs = []
for i, x in enumerate(integrated_fact_metadata):
    if x['metadata']['general_observation'] == query:
        query_idxs.append(i)
        if x['metadata']['category'] not in ('anatomical finding', 'disease'):
            print(x)
            print()
len(query_idxs)

{'fact': 'no pneumothorax after pacemaker placement', 'metadata': {'reason': 'The statement indicates the absence of a pneumothorax, which is a significant observation following a medical procedure. This finding can be visually confirmed on a chest X-ray. It suggests that the procedure was successful without complications.', 'too_noisy_or_irrelevant': 'no', 'visually_observable': 'yes', 'category': 'procedure', 'abnormality_status': 'completely normal or healthy', 'anatomical_location': 'chest', 'general_observation': 'no pneumothorax'}, 'extraction_method': 't5-small-finetuned'}

{'fact': 'no pneumothorax after pigtail placement', 'metadata': {'reason': "The statement indicates the absence of a pneumothorax, which is a significant observation following a medical procedure. It suggests that the pigtail placement of the device has been performed, which is relevant to the assessment of the patient's condition. This finding can be visually confirmed on a chest X-ray.", 'too_noisy_or_irrel

245

In [439]:
from collections import Counter

In [440]:
Counter(x['metadata']['category'] for x in integrated_fact_metadata)

Counter({'anatomical finding': 396065,
         'disease': 37122,
         'technical assessment': 53424,
         'tubes and lines': 83396,
         'device': 35156,
         'procedure': 160,
         'unknown': 83,
         'health status': 14,
         'symptom': 226,
         '': 413,
         'foreign body': 5,
         'surgical procedure': 3,
         'complication': 2,
         'treatment': 1,
         'surgery': 2,
         'normal': 1,
         'tubes and lines unchanged': 1})

In [441]:
Counter(x['metadata']['health status'] for x in integrated_fact_metadata)

Counter({'ambiguous': 41261,
         'abnormal': 315390,
         'unknown': 172249,
         'normal': 76889,
         'resolved': 5,
         'improved': 23,
         '': 257})

In [471]:
# _VALID_JSON_OBJECT_REGEX = re.compile(r"\{\s*\"reason\"\s*:\s*\"[^\"]*\"\s*,\s*\"too\_noisy\_or\_irrelevant\"\s*:\s*\"[^\"]*\"\s*,\s*\"visually\_observable\"\s*:\s*\"[^\"]*\"\s*,\s*\"category\"\s*:\s*\"[^\"]*\"\s*,\s*\"abnormality\_status\"\s*:\s*\"[^\"]*\"\s*,\s*\"anatomical\_location\?\"\s*:\s*\"[^\"]*\"\s*,\s*\"general\_observation\"\s*:\s*\"[^\"]*\"\s*\}")

_VALID_JSON_OBJECT_REGEX = re.compile(
    r'\{\s*"reason"\s*:\s*"[^"]*"\s*,\s*"too_noisy_or_irrelevant"\s*:\s*"[^"]*"\s*,\s*"visually_observable"\s*:\s*"[^"]*"\s*,\s*"category"\s*:\s*"[^"]*"\s*,\s*"abnormality_status"\s*:\s*"[^"]*"\s*,\s*"anatomical_location"\s*:\s*"[^"]*"\s*,\s*"general_observation"\s*:\s*"[^"]*"\s*\}',
    re.DOTALL
)

In [483]:
match = _VALID_JSON_OBJECT_REGEX.search("""{
  "reason": "The statement describes the presence of surgical clips, which are a visual finding on a chest X-ray. These clips indicate a previous surgical procedure, specifically an axillary dissection. This information is relevant for assessing the patient's surgical history and any potential implications for current health.",
  "too_noisy_or_irrelevant": "no",
  "visually_observable": "yes",
  "category": "device",
  "abnormality_status": "does not apply",
  "anatomical_location": "axilla",
  "general_observation": "surgical clips present"
}""")
if match:
    print(match.group(0))

{
  "reason": "The statement describes the presence of surgical clips, which are a visual finding on a chest X-ray. These clips indicate a previous surgical procedure, specifically an axillary dissection. This information is relevant for assessing the patient's surgical history and any potential implications for current health.",
  "too_noisy_or_irrelevant": "no",
  "visually_observable": "yes",
  "category": "device",
  "abnormality_status": "does not apply",
  "anatomical_location": "axilla",
  "general_observation": "surgical clips present"
}


In [224]:
import re
# _VALID_JSON_OBJECT_REGEX = re.compile(
#     r'\{\s*"reason"\s*:\s*"[^"]*"\s*,\s*"ruled_out_abnormalities"\s*:\s*\{\s*(?:"[^"]*"\s*:\s*\[\s*"[^"]*"(?:\s*,\s*"[^"]*")*\s*\]\s*,?\s*)*\}\s*\}',
#     re.DOTALL
# )
_VALID_JSON_OBJECT_REGEX = re.compile(
    r'\{\s*"reason"\s*:\s*"[^"]*"\s*,\s*"ruled_out_abnormalities"\s*:\s*\{\s*(?:"[^"]*"\s*:\s*\[\s*(?:"[^"]*"(?:\s*,\s*"[^"]*")*)?\s*\]\s*,?\s*)*\}\s*\}',
    re.DOTALL
)

In [230]:
input_string = """

```json
{
  "reason": "The report describes various post-surgical findings and the presence of medical devices, but also explicitly rules out certain conditions. 'No evidence of CHF' and 'no effusion seen on the frontal view' rule out congestive heart failure and pleural effusion, respectively. 'No new consolidation' and 'no pneumothorax detected' rule out the development of new lung consolidation and the presence of pneumothorax. However, 'mild CHF' is mentioned, which indicates that while there is no evidence of acute or severe congestive heart failure, there is a mild form present, so CHF is not completely ruled out.",
  "ruled_out_abnormalities": {
    "no evidence of CHF": [],
    "no effusion seen on the frontal view": ["pleural effusion"],
    "no new consolidation": ["lung consolidation"],
    "no pneumothorax detected": ["pneumothorax"]
  }
}
```
"""
_VALID_JSON_OBJECT_REGEX.search(input_string).group()

'{\n  "reason": "The report describes various post-surgical findings and the presence of medical devices, but also explicitly rules out certain conditions. \'No evidence of CHF\' and \'no effusion seen on the frontal view\' rule out congestive heart failure and pleural effusion, respectively. \'No new consolidation\' and \'no pneumothorax detected\' rule out the development of new lung consolidation and the presence of pneumothorax. However, \'mild CHF\' is mentioned, which indicates that while there is no evidence of acute or severe congestive heart failure, there is a mild form present, so CHF is not completely ruled out.",\n  "ruled_out_abnormalities": {\n    "no evidence of CHF": [],\n    "no effusion seen on the frontal view": ["pleural effusion"],\n    "no new consolidation": ["lung consolidation"],\n    "no pneumothorax detected": ["pneumothorax"]\n  }\n}'

In [76]:
# q = """subtle increased opacity in the right lung base. clinical situation. recommend close interval follow-up. developing pneumonia is possible. right PICC line has been removed. nasogastric tube in unchanged position. unchanged appearance of the cardiac silhouette. unchanged appearance of the hilar structures. unchanged appearance of the lung parenchyma. no pneumonia"""
q = 'escape of previous large right pleural air collection'
for x in integrated_report_facts:
#     if x['fact_based_report'] == q:
#         break
    if q in x['fact_based_report']:
        break

In [77]:
print(read_txt(x['path']))

                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)
 
 INDICATION:  ___ year old man with sob s/p pleurodesis  // any ptx?      any
 ptx?
 
 IMPRESSION: 
 
 Compared to chest radiographs ___ through ___.
 
 The possibility of small bilateral pneumothorax is impossible to exclude given
 the severity of stable but pronounced subcutaneous emphysema and
 pneumomediastinum, all of which is attributable to the escape of previous
 large right pleural air collection with placement of the right basal pigtail
 pleural drainage catheter.  Upper lobe predominant emphysema is so severe that
 bibasilar opacification could be due to edema alone, more pronounced on the
 right because of its recent Re expansion.  Pneumonia and pulmonary hemorrhage
 are not excluded.
 
 Heart size normal.



In [218]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 0 \
--num_samples 1000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 100000000 \
--max_tokens_per_request 500 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4-1106-preview" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-11 23:01:10,539 - INFO - Total number of fact based reports: 211226
2025-01-11 23:01:10,594 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-11 23:01:10,668 - INFO - Done loading cached sorted sentences.
2025-01-11 23:01:10,668 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior inferior portion of the T5 vertebral body. anterior epidural enhancement from T5-T7. suspicion for small component of epidural abscess. exaggerated kyphotic angulation at the T6/T7 level. small amount of retropulsion focall

2025-01-11 23:01:13,410 - INFO - HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
2025-01-11 23:01:13,412 - INFO - Creating batch with input file ID file-AbXtAx8RZcLEqfMbm2aJ83
2025-01-11 23:01:14,049 - INFO - HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"
2025-01-11 23:01:14,055 - INFO - Batch object: Batch(id='batch_67832269d0f081909a56366025a52656', completion_window='24h', created_at=1736647273, endpoint='/v1/chat/completions', input_file_id='file-AbXtAx8RZcLEqfMbm2aJ83', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736733673, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-11 23:01:14,056 - INFO - Saving batch object to /home/pamessina/medvqa-workspa

In [234]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-AbXtAx8RZcLEqfMbm2aJ83" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4-1106-preview" \
--use_batch_api

2025-01-11 23:38:20,465 - INFO - Checking status of batch with input file ID file-AbXtAx8RZcLEqfMbm2aJ83
2025-01-11 23:38:20,864 - INFO - Batch object: Batch(id='batch_67832269d0f081909a56366025a52656', completion_window='24h', created_at=1736647273, endpoint='/v1/chat/completions', input_file_id='file-AbXtAx8RZcLEqfMbm2aJ83', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736733673, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-11 23:38:22,032 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_67832269d0f081909a56366025a52656 "HTTP/1.1 200 OK"
2025-01-11 23:38:22,040 - INFO - Batch: Batch(id='batch_67832269d0f081909a56366025a52656', completion_window='24h', created_at=1736647273, e

    "skeletal structures of the thorax within normal limits": ["rib fracture", "costochondral separation"],
    "unremarkable rib structures including rib one": ["first rib fracture"],
    "unremarkable rib structures including rib two": ["second rib fracture"],
    "unremarkable rib structures including rib three": ["third rib fracture"],
    "unremarkable rib structures including rib four": ["fourth rib fracture"],
    "unremarkable rib structures including rib five": ["fifth rib fracture"],
    "unremarkable rib structures including rib six": ["sixth rib fracture"],
    "unremarkable rib structures including rib seven": ["seventh rib fracture"],
    "unremarkable rib structures including rib eight": ["eighth rib fracture"],
    "unremarkable rib structures including rib nine": ["ninth rib fracture"],
    "unremarkable rib structures including rib ten": ["tenth rib fracture"],
    "unremarkable rib structures including rib eleven": ["eleventh rib fracture"],
    "unremarkable rib str

In [248]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 0 \
--num_samples 10000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 100000000 \
--max_tokens_per_request 700 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4-1106-preview" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-11 23:54:56,877 - INFO - Loaded 999 already processed queries from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4-1106-preview_fact_based_report_to_negative_facts.jsonl
2025-01-11 23:55:00,586 - INFO - Total number of fact based reports: 211226
2025-01-11 23:55:00,640 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-11 23:55:00,713 - INFO - Done loading cached sorted sentences.
2025-01-11 23:55:00,713 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the ante

2025-01-11 23:55:01,028 - INFO - Saving batch API requests to /home/pamessina/medvqa-workspace/tmp/mimiccxr/openai/batch_api_requests_20250111_235501.jsonl
2025-01-11 23:55:04,708 - INFO - HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
2025-01-11 23:55:04,710 - INFO - Creating batch with input file ID file-1yWDvxryqSg1iv1QUMvF4Z
2025-01-11 23:55:05,737 - INFO - HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"
2025-01-11 23:55:05,744 - INFO - Batch object: Batch(id='batch_67832f09889081908b221f6b09a90912', completion_window='24h', created_at=1736650505, endpoint='/v1/chat/completions', input_file_id='file-1yWDvxryqSg1iv1QUMvF4Z', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736736905, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=Non

In [252]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-1yWDvxryqSg1iv1QUMvF4Z" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4-1106-preview" \
--use_batch_api

2025-01-12 02:05:46,610 - INFO - Checking status of batch with input file ID file-1yWDvxryqSg1iv1QUMvF4Z
2025-01-12 02:05:47,004 - INFO - Batch object: Batch(id='batch_67832f09889081908b221f6b09a90912', completion_window='24h', created_at=1736650505, endpoint='/v1/chat/completions', input_file_id='file-1yWDvxryqSg1iv1QUMvF4Z', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736736905, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-12 02:05:47,883 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_67832f09889081908b221f6b09a90912 "HTTP/1.1 200 OK"
2025-01-12 02:05:47,892 - INFO - Batch: Batch(id='batch_67832f09889081908b221f6b09a90912', completion_window='24h', created_at=1736650505, e

In [250]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 10000 \
--num_samples 40000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 20000000 \
--max_tokens_per_request 700 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-12 00:05:35,613 - INFO - Total number of fact based reports: 211226
2025-01-12 00:05:35,666 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-12 00:05:35,738 - INFO - Done loading cached sorted sentences.
2025-01-12 00:05:35,738 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior inferior portion of the T5 vertebral body. anterior epidural enhancement from T5-T7. suspicion for small component of epidural abscess. exaggerated kyphotic angulation at the T6/T7 level. small amount of retropulsion focall

In [259]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-6duoksM7RNQg3mSWdXpBtM" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-12 09:25:33,140 - INFO - Checking status of batch with input file ID file-6duoksM7RNQg3mSWdXpBtM
2025-01-12 09:25:33,532 - INFO - Batch object: Batch(id='batch_6783318b27108190bd0b17ad15b25a3c', completion_window='24h', created_at=1736651147, endpoint='/v1/chat/completions', input_file_id='file-6duoksM7RNQg3mSWdXpBtM', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736737547, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-12 09:25:34,599 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6783318b27108190bd0b17ad15b25a3c "HTTP/1.1 200 OK"
2025-01-12 09:25:34,607 - INFO - Batch: Batch(id='batch_6783318b27108190bd0b17ad15b25a3c', completion_window='24h', created_at=1736651147, e

In [251]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 50000 \
--num_samples 40000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 20000000 \
--max_tokens_per_request 700 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-12 00:09:54,842 - INFO - Total number of fact based reports: 211226
2025-01-12 00:09:54,895 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-12 00:09:54,967 - INFO - Done loading cached sorted sentences.
2025-01-12 00:09:54,968 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior inferior portion of the T5 vertebral body. anterior epidural enhancement from T5-T7. suspicion for small component of epidural abscess. exaggerated kyphotic angulation at the T6/T7 level. small amount of retropulsion focall

In [260]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-NC5zixo1R4XAAoZWetgtQh" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-12 09:26:10,833 - INFO - Checking status of batch with input file ID file-NC5zixo1R4XAAoZWetgtQh
2025-01-12 09:26:11,222 - INFO - Batch object: Batch(id='batch_6783328dbb4481909732fb30c16176eb', completion_window='24h', created_at=1736651406, endpoint='/v1/chat/completions', input_file_id='file-NC5zixo1R4XAAoZWetgtQh', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736737806, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-12 09:26:11,741 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6783328dbb4481909732fb30c16176eb "HTTP/1.1 200 OK"
2025-01-12 09:26:11,749 - INFO - Batch: Batch(id='batch_6783328dbb4481909732fb30c16176eb', completion_window='24h', created_at=1736651406, e

In [1]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 90000 \
--num_samples 30000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 20000000 \
--max_tokens_per_request 700 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-12 12:15:44,330 - INFO - Loaded 79998 already processed queries from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_fact_based_report_to_negative_facts.jsonl
2025-01-12 12:15:48,916 - INFO - Total number of fact based reports: 211226
2025-01-12 12:15:48,991 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-12 12:15:49,103 - INFO - Done loading cached sorted sentences.
2025-01-12 12:15:49,103 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior infer

In [31]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-MMLj91ppHP8pu1SUhkmHwk" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-12 14:15:11,539 - INFO - Checking status of batch with input file ID file-MMLj91ppHP8pu1SUhkmHwk
2025-01-12 14:15:11,936 - INFO - Batch object: Batch(id='batch_6783dcae2fb08190a03b1343b4d3b29b', completion_window='24h', created_at=1736694958, endpoint='/v1/chat/completions', input_file_id='file-MMLj91ppHP8pu1SUhkmHwk', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736781358, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-12 14:15:12,820 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6783dcae2fb08190a03b1343b4d3b29b "HTTP/1.1 200 OK"
2025-01-12 14:15:12,828 - INFO - Batch: Batch(id='batch_6783dcae2fb08190a03b1343b4d3b29b', completion_window='24h', created_at=1736694958, e

In [37]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--offset 120000 \
--num_samples 30000 \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--max_requests_per_minute 9000 \
--max_tokens_per_minute 20000000 \
--max_tokens_per_request 700 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "extract negative facts from fact-based reports"

2025-01-13 04:18:02,440 - INFO - Loaded 109998 already processed queries from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_fact_based_report_to_negative_facts.jsonl
2025-01-13 04:18:05,932 - INFO - Total number of fact based reports: 211226
2025-01-13 04:18:05,987 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(211226,75856766).pkl...
2025-01-13 04:18:06,060 - INFO - Done loading cached sorted sentences.
2025-01-13 04:18:06,060 - INFO - First sentence: Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior infe

In [39]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_reports_with_openai.py \
--batch_input_file_id "file-HG5ySPEVX4Yc3cCPVMco4p" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-13 05:50:04,761 - INFO - Checking status of batch with input file ID file-HG5ySPEVX4Yc3cCPVMco4p
2025-01-13 05:50:05,381 - INFO - Batch object: Batch(id='batch_6784be36e5f481908a634f72bf6be91d', completion_window='24h', created_at=1736752694, endpoint='/v1/chat/completions', input_file_id='file-HG5ySPEVX4Yc3cCPVMco4p', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736839094, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'extract negative facts from fact-based reports'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-13 05:50:06,407 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6784be36e5f481908a634f72bf6be91d "HTTP/1.1 200 OK"
2025-01-13 05:50:06,416 - INFO - Batch: Batch(id='batch_6784be36e5f481908a634f72bf6be91d', completion_window='24h', created_at=1736752694, e

In [11]:
tmp1 = load_jsonl('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4-1106-preview_fact_based_report_to_negative_facts.jsonl')

In [13]:
len(tmp1)

10000

In [40]:
tmp2 = load_jsonl('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_fact_based_report_to_negative_facts.jsonl')

In [41]:
len(tmp2)

139995

In [94]:
q = "avoidance of Ms. of the right main bronchus"
for x in tmp1 + tmp2:
    if q in x['metadata']['query']:
        print(x)
        break

{'metadata': {'query': 'unchanged low position of the endotracheal tube. tube needs to be withdrawn by about 2 cm. avoidance of Ms. of the right main bronchus. course of the nasogastric tube is unchanged. unchanged very low lung volumes. retrocardiac atelectasis. right basilar atelectasis. unchanged mild to moderate pulmonary edema'}, 'parsed_response': {'reason': 'The report describes the current state of various tubes and conditions within the chest, but does not explicitly rule out any abnormalities. Instead, it notes the presence of certain conditions such as atelectasis and pulmonary edema, and provides recommendations for the adjustment of the endotracheal tube. There are no statements that definitively indicate the absence of any abnormalities.', 'ruled_out_abnormalities': {}}}


In [47]:
abnormalities = set()
for x in tmp1 + tmp2:
    for k, v in x['parsed_response']['ruled_out_abnormalities'].items():
        assert isinstance(v, list)
        abnormalities.update(v)
        if 'PICC line in right atrium' in v:
            print(x)
abnormalities = list(abnormalities)
abnormalities.sort()
len(abnormalities)

11530

In [43]:
abnormalities[:100]

['AC joint arthritis',
 'AC joint dislocation',
 'AICD displacement',
 'AICD lead dislodgement',
 'AICD lead fracture',
 'AICD malfunction',
 'Acute Respiratory Distress Syndrome',
 'Acute Respiratory Distress Syndrome (ARDS)',
 'C-spine hardware displacement',
 'C-spine hardware failure',
 'CHF focal consolidation',
 'Dobbhoff catheter coiling',
 'Dobbhoff catheter complication',
 'Dobbhoff catheter kinking',
 'Dobbhoff catheter malposition',
 'Dobbhoff catheter obstruction',
 'Dobbhoff catheter-related complication',
 'Dobbhoff line coiling',
 'Dobbhoff line malposition',
 'Dobbhoff line placement complication',
 'Dobbhoff line-associated injury',
 'Dobbhoff tube coiling',
 'Dobbhoff tube complication',
 'Dobbhoff tube malposition',
 'Dobbhoff tube misplacement',
 'Dobbhoff tube misplacement in the chest',
 'Dobbhoff tube misplacement in the neck',
 'Dobbhoff tube presence',
 'Dobbhoff tube-related complication',
 'Dobhoff catheter complication',
 'Dobhoff tube in the respiratory tra

In [19]:
import json
json.dumps(tmp1[0])

'{"metadata": {"query": "Destruction of the T6/T7 interspace. Increased T2/STIR signal at the T6/T7 interspace. Postcontrast enhancement at the T6/T7 interspace. Partial destruction of the T6 inferior end plate. Partial destruction of the T7 superior endplate. Loss of anterior vertebral body height at T6/T7. Discitis/osteomyelitis complex at T6/T7. prominent prevertebral enhancing soft tissue affecting the anterior inferior portion of the T5 vertebral body. anterior epidural enhancement from T5-T7. suspicion for small component of epidural abscess. exaggerated kyphotic angulation at the T6/T7 level. small amount of retropulsion focally contacting the cord. without significant canal stenosis. normal cord signal. multilevel non-critical degenerative changes. Multilevel Tarlov cysts. normal heart size. mild tortuosity of the thoracic aorta. unremarkable hilar contours. clear lungs. normal pleural surfaces. posterior thickening. healed left posterolateral rib fractures. midthoracic disc sp

In [24]:
q = 'cardiopneumothorax'
for x in tmp1:
    if q in json.dumps(x):
        print(x)
        break

In [44]:
tmp = tmp2

In [46]:
from pprint import pprint
i = -2
print(tmp[i]['metadata']['query'])
print()
pprint(tmp[i]['parsed_response'])

single frontal view of the chest. no significant changes compared to prior exam. postsurgical changes at the right lung base. stable calcified granuloma in the left lung base. small bilateral pleural effusions. enlarged heart size is unchanged. no evidence of pneumothorax

{'reason': 'The report describes a comparison with a prior exam, noting '
           'stability in certain findings. It mentions postsurgical changes '
           'and a stable granuloma, indicating no new developments. The '
           'presence of small pleural effusions and an unchanged enlarged '
           'heart size are noted. Pneumothorax is explicitly ruled out.',
 'ruled_out_abnormalities': {'no evidence of pneumothorax': ['pneumothorax']}}


In [82]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 0 \
--num_samples 5000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 16:16:04,810 - INFO - Total number of unique sentences: 592009
2025-01-13 16:16:04,948 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(592009,27712466).pkl...
2025-01-13 16:16:05,026 - INFO - Done loading cached sorted sentences.
2025-01-13 16:16:05,027 - INFO - First sentence: A BENIGN CALCIFICATION IN LUNG OR RIB, OR VESSEL, EITHER ON END OR CROSSING in the right lower lung
2025-01-13 16:16:05,027 - INFO - Last sentence:  
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 876627
self.cache["embeddings"].shape = (876627, 128)
Loading cached kmeans labels from /mnt/data/pamessina_folder_backup_15_1

In [85]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-GYg6WAzVz2DBFCHLye86Kq" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-13 17:34:25,887 - INFO - Checking status of batch with input file ID file-GYg6WAzVz2DBFCHLye86Kq
2025-01-13 17:34:26,324 - INFO - Batch object: Batch(id='batch_6785667d41a48190aab6589870f2d3e2', completion_window='24h', created_at=1736795773, endpoint='/v1/chat/completions', input_file_id='file-GYg6WAzVz2DBFCHLye86Kq', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736882173, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-13 17:34:27,310 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6785667d41a48190aab6589870f2d3e2 "HTTP/1.1 200 OK"
2025-01-13 17:34:27,318 - INFO - Batch: Batch(id='batch_6785667d41a48190aab6589870f2d3e2', completion_window='24h', created_at=1736795773, endpoint='/v1/chat/co

In [6]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 0 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 350 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:29:43,986 - INFO - Loaded 4994 already processed queries from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl
2025-01-13 18:29:49,955 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:29:50,252 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:29:50,433 - INFO - Done loading cached sorted sentences.
2025-01-13 18:29:50,433 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:29:50,433 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta

In [15]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-BidVjgMYq8xRF5obymmxcZ" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o" \
--use_batch_api

2025-01-14 10:17:44,306 - INFO - Checking status of batch with input file ID file-BidVjgMYq8xRF5obymmxcZ
2025-01-14 10:17:44,746 - INFO - Batch object: Batch(id='batch_678585e63c9481908ff09e1c4b840103', completion_window='24h', created_at=1736803814, endpoint='/v1/chat/completions', input_file_id='file-BidVjgMYq8xRF5obymmxcZ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890214, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:17:45,622 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_678585e63c9481908ff09e1c4b840103 "HTTP/1.1 200 OK"
2025-01-14 10:17:45,630 - INFO - Batch: Batch(id='batch_678585e63c9481908ff09e1c4b840103', completion_window='24h', created_at=1736803814, endpoint='/v1/chat/co

In [7]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 50000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:31:54,201 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:31:54,505 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:31:54,681 - INFO - Done loading cached sorted sentences.
2025-01-13 18:31:54,681 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:31:54,681 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [16]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-3x6mWvMFfj4YFPVv7dAUFi" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:18:44,964 - INFO - Checking status of batch with input file ID file-3x6mWvMFfj4YFPVv7dAUFi
2025-01-14 10:18:45,398 - INFO - Batch object: Batch(id='batch_6785866798c8819085c770299ac6ecb2', completion_window='24h', created_at=1736803943, endpoint='/v1/chat/completions', input_file_id='file-3x6mWvMFfj4YFPVv7dAUFi', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890343, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:18:46,141 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6785866798c8819085c770299ac6ecb2 "HTTP/1.1 200 OK"
2025-01-14 10:18:46,149 - INFO - Batch: Batch(id='batch_6785866798c8819085c770299ac6ecb2', completion_window='24h', created_at=1736803943, endpoint='/v1/chat/co

In [8]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 100000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:32:49,521 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:32:49,826 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:32:50,006 - INFO - Done loading cached sorted sentences.
2025-01-13 18:32:50,006 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:32:50,006 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [17]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-6jngwynFipUWXjyyxWsNim" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:19:54,639 - INFO - Checking status of batch with input file ID file-6jngwynFipUWXjyyxWsNim
2025-01-14 10:19:55,069 - INFO - Batch object: Batch(id='batch_678586991ab08190a080409823ada87d', completion_window='24h', created_at=1736803993, endpoint='/v1/chat/completions', input_file_id='file-6jngwynFipUWXjyyxWsNim', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890393, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:19:56,166 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_678586991ab08190a080409823ada87d "HTTP/1.1 200 OK"
2025-01-14 10:19:56,173 - INFO - Batch: Batch(id='batch_678586991ab08190a080409823ada87d', completion_window='24h', created_at=1736803993, endpoint='/v1/chat/co

In [9]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 150000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:33:25,519 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:33:25,820 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:33:25,997 - INFO - Done loading cached sorted sentences.
2025-01-13 18:33:25,997 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:33:25,997 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [18]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-Jo6ZgNM44ckYBSz5HCWYm8" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:21:13,778 - INFO - Checking status of batch with input file ID file-Jo6ZgNM44ckYBSz5HCWYm8
2025-01-14 10:21:14,215 - INFO - Batch object: Batch(id='batch_678586bcb1f4819080cad25c76e64bc8', completion_window='24h', created_at=1736804028, endpoint='/v1/chat/completions', input_file_id='file-Jo6ZgNM44ckYBSz5HCWYm8', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890428, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:21:14,604 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_678586bcb1f4819080cad25c76e64bc8 "HTTP/1.1 200 OK"
2025-01-14 10:21:14,612 - INFO - Batch: Batch(id='batch_678586bcb1f4819080cad25c76e64bc8', completion_window='24h', created_at=1736804028, endpoint='/v1/chat/co

In [10]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 200000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:34:01,260 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:34:01,563 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:34:01,744 - INFO - Done loading cached sorted sentences.
2025-01-13 18:34:01,744 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:34:01,744 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [67]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-L6b2nzes5YxVi77JugQdhv" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 18:41:51,781 - INFO - Checking status of batch with input file ID file-L6b2nzes5YxVi77JugQdhv
2025-01-14 18:41:52,296 - INFO - Batch object: Batch(id='batch_678586e069d881909a72ddff19b5e8bb', completion_window='24h', created_at=1736804064, endpoint='/v1/chat/completions', input_file_id='file-L6b2nzes5YxVi77JugQdhv', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890464, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 18:41:53,098 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_678586e069d881909a72ddff19b5e8bb "HTTP/1.1 200 OK"
2025-01-14 18:41:53,106 - INFO - Batch: Batch(id='batch_678586e069d881909a72ddff19b5e8bb', completion_window='24h', created_at=1736804064, endpoint='/v1/chat/co

In [11]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 250000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:34:36,686 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:34:36,984 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:34:37,165 - INFO - Done loading cached sorted sentences.
2025-01-13 18:34:37,166 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:34:37,166 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [20]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-XckYHPzy5p7BALNAeBJi7H" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:22:57,353 - INFO - Checking status of batch with input file ID file-XckYHPzy5p7BALNAeBJi7H
2025-01-14 10:22:57,793 - INFO - Batch object: Batch(id='batch_67858704e4588190b5494657e609ddcd', completion_window='24h', created_at=1736804101, endpoint='/v1/chat/completions', input_file_id='file-XckYHPzy5p7BALNAeBJi7H', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890501, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:22:59,203 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_67858704e4588190b5494657e609ddcd "HTTP/1.1 200 OK"
2025-01-14 10:22:59,211 - INFO - Batch: Batch(id='batch_67858704e4588190b5494657e609ddcd', completion_window='24h', created_at=1736804101, endpoint='/v1/chat/co

In [12]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 300000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:35:13,304 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:35:13,606 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:35:13,780 - INFO - Done loading cached sorted sentences.
2025-01-13 18:35:13,780 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:35:13,780 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [21]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-9mRqScmJC1mgzRFjqBpaRi" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:23:24,799 - INFO - Checking status of batch with input file ID file-9mRqScmJC1mgzRFjqBpaRi
2025-01-14 10:23:25,228 - INFO - Batch object: Batch(id='batch_6785872927b08190894ded7a36e6b129', completion_window='24h', created_at=1736804137, endpoint='/v1/chat/completions', input_file_id='file-9mRqScmJC1mgzRFjqBpaRi', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890537, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:23:25,762 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6785872927b08190894ded7a36e6b129 "HTTP/1.1 200 OK"
2025-01-14 10:23:25,770 - INFO - Batch: Batch(id='batch_6785872927b08190894ded7a36e6b129', completion_window='24h', created_at=1736804137, endpoint='/v1/chat/co

In [13]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 350000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:35:49,543 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:35:49,866 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:35:50,050 - INFO - Done loading cached sorted sentences.
2025-01-13 18:35:50,050 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:35:50,050 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [22]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-GKX6NbK9VEraK4tB6gRD9K" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:25:08,849 - INFO - Checking status of batch with input file ID file-GKX6NbK9VEraK4tB6gRD9K
2025-01-14 10:25:09,307 - INFO - Batch object: Batch(id='batch_6785874d99e4819091474be73e1fe251', completion_window='24h', created_at=1736804173, endpoint='/v1/chat/completions', input_file_id='file-GKX6NbK9VEraK4tB6gRD9K', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890573, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:25:09,707 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_6785874d99e4819091474be73e1fe251 "HTTP/1.1 200 OK"
2025-01-14 10:25:09,715 - INFO - Batch: Batch(id='batch_6785874d99e4819091474be73e1fe251', completion_window='24h', created_at=1736804173, endpoint='/v1/chat/co

In [14]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--offset 400000 \
--num_samples 50000 \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--max_requests_per_minute 20000 \
--max_tokens_per_minute 10000000 \
--max_tokens_per_request 350 \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api \
--batch_description "sentence to negative facts"

2025-01-13 18:37:50,333 - INFO - Total number of unique sentences: 1269593
2025-01-13 18:37:50,634 - INFO - Loading cached sorted sentences from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/sorted_sentences(1269593,86367250).pkl...
2025-01-13 18:37:50,811 - INFO - Done loading cached sorted sentences.
2025-01-13 18:37:50,811 - INFO - First sentence: l CP ANGLE OBSCURED BUT PT HAD SIMILAR APPEARANCE ON SCOUTS FOR PRIOR CT L-SPINE AT WHICH POINT NO PLEURAL EFFUSION WAS EVIDENT- THUS THIS MAY REPRESENT A PERICARDIAL FAT PAD- THOUGH CANNOT DEFINITIVELY RULE OUT LAYERING LEFT EFFUSION.
2025-01-13 18:37:50,811 - INFO - Last sentence: .
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["

In [23]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_openai.py \
--batch_input_file_id "file-UNzHT5PdnQeAfMNjNLurxv" \
--logging_level "INFO" \
--api_key_name "OPENAI_API_KEY_1" \
--openai_model_name "gpt-4o-mini" \
--use_batch_api

2025-01-14 10:25:28,364 - INFO - Checking status of batch with input file ID file-UNzHT5PdnQeAfMNjNLurxv
2025-01-14 10:25:28,801 - INFO - Batch object: Batch(id='batch_678587c5b98881908765b9cf29557951', completion_window='24h', created_at=1736804293, endpoint='/v1/chat/completions', input_file_id='file-UNzHT5PdnQeAfMNjNLurxv', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736890693, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'sentence to negative facts'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2025-01-14 10:25:29,887 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_678587c5b98881908765b9cf29557951 "HTTP/1.1 200 OK"
2025-01-14 10:25:29,895 - INFO - Batch: Batch(id='batch_678587c5b98881908765b9cf29557951', completion_window='24h', created_at=1736804293, endpoint='/v1/chat/co

In [24]:
tmp1 = load_jsonl('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl')

In [68]:
tmp2 = load_jsonl('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl')

In [69]:
len(tmp1), len(tmp2)

(50436, 398547)

In [62]:
import json
q = "enlarged epipericardial fat pad is preserved"
for x in tmp2:
    if q in json.dumps(x):
        print(x)

{'metadata': {'query': 'Again, an enlarged epipericardial fat pad is preserved.'}, 'parsed_response': {'reason': 'The statement indicates that the enlarged epipericardial fat pad is still present, suggesting that it has not resolved or changed. There are no indications of any abnormalities being ruled out in this context.', 'ruled_out_abnormalities': {}}}
{'metadata': {'query': 'enlarged epipericardial fat pad is preserved'}, 'parsed_response': {'reason': "The phrase 'is preserved' indicates that the enlarged epipericardial fat pad remains unchanged and does not suggest any abnormalities. Therefore, it rules out any pathological conditions related to the epipericardial fat pad.", 'ruled_out_abnormalities': {'enlarged epipericardial fat pad is preserved': ['epipericardial fat pad abnormality', 'epipericardial fat pad pathology']}}}


In [37]:
tmp2[-1]

{'metadata': {'query': 'difficult to exclude right base consolidation'},
 'parsed_response': {'reason': 'The statement indicates uncertainty regarding the presence of consolidation at the right base, suggesting that it cannot be definitively ruled out. Therefore, no specific abnormalities can be excluded based on this information.',
  'ruled_out_abnormalities': {}}}

In [45]:
aux = [x for x in tmp2 if len(x['parsed_response']['ruled_out_abnormalities']) > 0]
len(aux)

43576

In [44]:
aux[100]

{'metadata': {'query': 'satisfactory reexpansion following unclamping'},
 'parsed_response': {'reason': 'The sentence indicates that the reexpansion process was successful after unclamping, suggesting that the area in question is now normal.',
  'ruled_out_abnormalities': {'satisfactory reexpansion': ['lung collapse',
    'atelectasis']}}}

In [95]:
integrated_sentence_facts[0]

{'sentence': 'Mild cardiomegaly is unchanged, but pulmonary edema has worsened, most pronounced in the right upper lobe.',
 'facts': ['mild cardiomegaly',
  'unchanged mild cardiomegaly',
  'worsened pulmonary edema',
  'most pronounced pulmonary edema in the right upper lobe'],
 'extraction_method': 't5-small-finetuned'}

In [110]:
q = 'satisfactory reexpansion following unclamping'
for x in integrated_sentence_facts:
    if q in x['facts']:
        print(x)
        break

{'sentence': 'Subsequent radiographs demonstrate satisfactory reexpansion following unclamping.', 'facts': ['satisfactory reexpansion following unclamping'], 'extraction_method': 'gpt-4-0613'}


In [71]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_transformer.py \
--preprocessed_queries_to_skip_filepaths \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl" \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl" \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--logging_level "INFO" \
--checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/seq2seq/20250114_184937_report2negative_facts(r2nf)_Seq2Seq(t5-small)" \
--batch_size 250 \
--num_beams 1 \
--max_length 250 \
--num_workers 4

2025-01-16 14:36:09,596 - INFO - Loading preprocessed queries to skip from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl
2025-01-16 14:36:10,230 - INFO - Loaded 50436 queries to skip
2025-01-16 14:36:10,260 - INFO - Total number of queries to skip: 50436
2025-01-16 14:36:10,260 - INFO - Loading preprocessed queries to skip from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl
2025-01-16 14:36:14,368 - INFO - Loaded 398547 queries to skip
2025-01-16 14:36:14,619 - INFO - Total number of queries to skip: 448573
2025-01-16 14:36:14,619 - INFO - Loading sentences from /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl
2025-01-16 14:36:20,446 - INFO - Found 429416 sentences to process
2025-01-16 14:36:20,480 - INFO - Example sentences to process:
2025-01-16 14:36:20,494 - 

 94%|████████████████████████████████████▌  | 1612/1718 [25:12<02:08,  1.21s/it]2025-01-16 15:01:47,676 - WARNING - Failed to process output text: "unchanged heart contours": ["cardiomegaly", "heart enlargement"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal surgical clips": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": ["mediastinal mass", "mediastinal widening"], "unchanged mediastinal contours": [", for sentence: The heart and mediastinal contours, along with mediastinal surgical clips and sternotomy wires, are unchanged., idx: 403251
2025-01-16 15:01:47,676 - WARNING - Exceptio

In [72]:
tmp = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl')

In [73]:
len(tmp)

429413

In [103]:
tmp[-45]

{'sentence': 'There is slight elevation of the left hemidiaphragm with blunting of the left costophrenic angle and atelectasis at the left lung base, similar to the prior film.',
 'ruled_out_abnormalities': {}}

In [104]:
tmp2 = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_15035.unprocessed.jsonl')

In [105]:
tmp2

['Left upper lobe pneumonia is improved, small left pleural effusion has resolved, small right pleural effusion is unchanged, and residual atelectasis is stable since .',
 'Right chest tube, left pigtail, NG tube, right subclavian, and NG tube all in appropriate positioning.',
 'The heart and mediastinal contours, along with mediastinal surgical clips and sternotomy wires, are unchanged.']

In [106]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_transformer.py \
--preprocessed_queries_to_skip_filepaths \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl" \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl" \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl" \
--integrated_sentence_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl" \
--logging_level "INFO" \
--checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/seq2seq/20250114_184937_report2negative_facts(r2nf)_Seq2Seq(t5-small)" \
--batch_size 250 \
--num_beams 5 \
--max_length 250 \
--num_workers 4

2025-01-16 15:26:26,306 - INFO - Loading preprocessed queries to skip from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl
2025-01-16 15:26:26,930 - INFO - Loaded 50436 queries to skip
2025-01-16 15:26:26,960 - INFO - Total number of queries to skip: 50436
2025-01-16 15:26:26,961 - INFO - Loading preprocessed queries to skip from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl
2025-01-16 15:26:31,007 - INFO - Loaded 398547 queries to skip
2025-01-16 15:26:31,265 - INFO - Total number of queries to skip: 448573
2025-01-16 15:26:31,265 - INFO - Loading preprocessed queries to skip from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl
2025-01-16 15:26:34,434 - INFO - Loaded 429413 queries to skip
2025-01-16 15:26:34,884 - INFO

In [107]:
load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_5_20250116_152645.jsonl')

[{'sentence': 'The heart and mediastinal contours, along with mediastinal surgical clips and sternotomy wires, are unchanged.',
  'ruled_out_abnormalities': {'unchanged heart contours': ['cardiomegaly',
    'heart enlargement'],
   'unchanged mediastinal contours': ['mediastinal mass',
    'mediastinal widening'],
   'unchanged mediastinal surgical clips': ['mediastinal mass',
    'mediastinal widening'],
   'unchanged sternotomy wires': ['sternotomy wire displacement',
    'sternotomy wire fracture']}},
 {'sentence': 'Right chest tube, left pigtail, NG tube, right subclavian, and NG tube all in appropriate positioning.',
  'ruled_out_abnormalities': {'appropriate positioning of right chest tube': ['malposition of right chest tube',
    'displacement of right chest tube'],
   'appropriate positioning of left pigtail': ['malposition of left pigtail',
    'displacement of left pigtail'],
   'appropriate positioning of NG tube': ['malposition of right subclavian tube',
    'malposition of

In [122]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_transformer.py \
--integrate_files \
--jsonl_filepaths_to_integrate \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl" \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl" \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl" \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_5_20250116_152645.jsonl" \
--extraction_methods \
"gpt-4o" \
"gpt-4o-mini" \
"t5-small-finetuned" \
"t5-small-finetuned"

Loading extracted negative facts from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl...
100%|██████████████████████████████████| 50436/50436 [00:00<00:00, 80554.53it/s]
Loading extracted negative facts from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl...
100%|████████████████████████████████| 398547/398547 [00:06<00:00, 58335.54it/s]
Loading extracted negative facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl...
100%|████████████████████████████████| 429413/429413 [00:09<00:00, 43760.86it/s]
Loading extracted negative facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_5_20250116_152645.jsonl...
100%|█████████████████████████████████

In [669]:
!python ../../scripts/mimiccxr/extract_negative_facts_from_report_sentences_with_transformer.py \
--integrate_files \
--jsonl_filepaths_to_integrate \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl" \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl" \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl" \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_5_20250116_152645.jsonl" \
--extraction_methods \
"gpt-4o" \
"gpt-4o-mini" \
"t5-small-finetuned" \
"t5-small-finetuned"

Loading extracted negative facts from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o_sentence_to_negative_facts.jsonl...
100%|██████████████████████████████████| 50436/50436 [00:00<00:00, 79538.22it/s]
Loading extracted negative facts from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4o-mini_sentence_to_negative_facts.jsonl...
100%|████████████████████████████████| 398547/398547 [00:06<00:00, 58225.32it/s]
Loading extracted negative facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_1_20250116_150357.jsonl...
100%|████████████████████████████████| 429413/429413 [00:09<00:00, 44004.95it/s]
Loading extracted negative facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/negative_facts_from_report_sentences_Seq2Seq(t5-small)_250_5_20250116_152645.jsonl...
100%|█████████████████████████████████

In [153]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157625400).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.65 \
--use_vindrcxr_facts

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 25
min_similarity_threshold: 0.65
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157625400).jsonl...
len(unique_negative_facts): 21243
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 1570498
self.cache["embeddings"].shape = (1570498, 128)
100%|████████████████████████████

In [166]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=25,sf=129898,gf=44029,nf=5972,minsimth=0.650)(hash=686,1961964868541064712).pkl"

-rw-rw-r-- 1 pamessina socvis 12M Jan 16 19:54 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=25,sf=129898,gf=44029,nf=5972,minsimth=0.650)(hash=686,1961964868541064712).pkl'


In [167]:
tmp = load_pickle("/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=25,sf=129898,gf=44029,nf=5972,minsimth=0.650)(hash=686,1961964868541064712).pkl")

In [168]:
tmp.keys()

dict_keys(['anchor_facts', 'preserved_general_facts', 'preserved_specific_facts', 'preserved_negative_facts', 'most_similar_to_specific_fact', 'most_similar_to_general_fact', 'most_similar_to_negative_fact'])

In [169]:
print(len(tmp['preserved_general_facts']))
print(len(tmp['preserved_specific_facts']))
print(len(tmp['preserved_negative_facts']))

44029
129898
5972


In [177]:
class_idx = tmp['anchor_facts'].index('aortic enlargement')
class_idx

0

In [178]:
idxs1 = [i for i, x in enumerate(tmp['most_similar_to_general_fact']) if x is not None and x[1] == class_idx]
idxs1.sort(key=lambda idx: tmp['most_similar_to_general_fact'][idx][0], reverse=True)

idxs2 = [i for i, x in enumerate(tmp['most_similar_to_specific_fact']) if x[1] == class_idx]
idxs2.sort(key=lambda idx: tmp['most_similar_to_specific_fact'][idx][0], reverse=True)

idxs3 = [i for i, x in enumerate(tmp['most_similar_to_negative_fact']) if x[1] == class_idx]
idxs3.sort(key=lambda idx: tmp['most_similar_to_negative_fact'][idx][0], reverse=True)

len(idxs1), len(idxs2), len(idxs3)

(1690, 6641, 308)

In [179]:
for i in range(0, min(100, len(idxs1))):
    i = idxs1[-i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_general_fact'][i][1]]
    fac = tmp['preserved_general_facts'][i]
    sim = tmp['most_similar_to_general_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9999999403953552	aortic enlargement (aortic enlargement)
0.2240704447031021	Sternotomy wires identified (aortic enlargement)
0.36848869919776917	possible hypervascularity (aortic enlargement)
0.42150241136550903	calcification in vascular structures (aortic enlargement)
0.4432164132595062	lymphadenopathy indicative of infection (aortic enlargement)
0.46396884322166443	possibility of dilated vessels (aortic enlargement)
0.4664706587791443	obscured hilar contours (aortic enlargement)
0.4679085314273834	prominent aortic knob unchanged (aortic enlargement)
0.4816696345806122	enlargement of the graft (aortic enlargement)
0.49418818950653076	unchanged aneurysmal dilatation (aortic enlargement)
0.5017157196998596	tortuous but normal caliber aorta (aortic enlargement)
0.5034435987472534	distortion of vascular architecture (aortic enlargement)
0.5075288414955139	possible aneurysm (aortic enlargement)
0.5127377510070801	persistent marked tortuosity (aortic enlargement)
0.5275207757949829	perihi

In [180]:
for i in range(0, min(100, len(idxs2))):
    i = idxs2[-i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_specific_fact'][i][1]]
    fac = tmp['preserved_specific_facts'][i]
    sim = tmp['most_similar_to_specific_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9999999403953552	aortic enlargement (aortic enlargement)
0.13645142316818237	areas of opacification above the right hemidiaphragmatic contour (aortic enlargement)
0.1393778920173645	milder hazy right perihilar opacification (aortic enlargement)
0.14035749435424805	rounded structure overlying the left upper quadrant (aortic enlargement)
0.14656345546245575	localized area of opacification in the left upper zone (aortic enlargement)
0.16093005239963531	radiopaque areas in the left upper quadrant (aortic enlargement)
0.16949015855789185	radiopaque densities projecting over left upper quadrant (aortic enlargement)
0.1708667278289795	radiopaque densities projecting over the left upper quadrant (aortic enlargement)
0.17104472219944	left perihilar round opacification less well seen (aortic enlargement)
0.17327085137367249	ring like densities within the left upper quadrant of the abdomen (aortic enlargement)
0.1902197152376175	lobulated density projecting over the left upper zone (aortic enla

In [172]:
for i in range(0, min(100, len(idxs3))):
    i = idxs3[-i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_negative_fact'][i][1]]
    fac = tmp['preserved_negative_facts'][i]
    sim = tmp['most_similar_to_negative_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

1.0	lung opacity (lung opacity)
0.3342297673225403	posterior opacification (lung opacity)
0.352603018283844	new focal opacity in the left lower lobe (lung opacity)
0.38320091366767883	right lower lung opacity (lung opacity)
0.39520886540412903	focal opacity in the left lower lobe (lung opacity)
0.41364115476608276	right lung base opacification (lung opacity)
0.4141662120819092	right lower lung opacification (lung opacity)
0.41453152894973755	right lower lung field opacity (lung opacity)
0.415157675743103	new left lung opacity (lung opacity)
0.4163133203983307	opacification at the right lung base (lung opacity)
0.41722071170806885	right lower lung zone opacity (lung opacity)
0.4286625385284424	right lower lung zone opacification (lung opacity)
0.435288667678833	parenchymal opacity in the left lower lobe (lung opacity)
0.43999823927879333	heterogeneous opacification in the left lower lobe (lung opacity)
0.44139403104782104	heterogeneous opacification in the lower lobe (lung opacity)
0.44

In [181]:
# Create embedding extractor
embedding_extractor = CachedTextEmbeddingExtractor(
    model_name='microsoft/BiomedVLP-CXR-BERT-specialized',
    model_checkpoint_folder_path='/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)',
    device='cuda',
)

checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']


In [184]:
anchor_embeddings = embedding_extractor.compute_text_embeddings(tmp['anchor_facts'], update_cache_on_disk=False)

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 105384.52it/s]


In [230]:
query = 'acute bone lesion'
query_embedding = embedding_extractor.compute_text_embeddings([query], update_cache_on_disk=False)[0]
idxs = rank_vectors_by_dot_product(anchor_embeddings, query_embedding)
for k in range(len(idxs)):
    print(tmp['anchor_facts'][idxs[k]], np.dot(anchor_embeddings[idxs[k]], query_embedding))

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8338.58it/s]

infiltration 0.6469321
nodule/mass 0.61805177
consolidation 0.53777665
lung opacity 0.5020517
clavicle fracture 0.48253575
enlarged pulmonary artery 0.44464284
edema 0.4213851
cardiomegaly 0.40852526
rib fracture 0.3983273
pleural thickening 0.3955556
emphysema 0.3933773
mediastinal shift 0.3665044
pulmonary fibrosis 0.3553869
copd 0.34237832
aortic enlargement 0.30483872
interstitial lung disease 0.23485985
calcification 0.15550865
pneumonia 0.1340104
lung cyst 0.123853184
lung tumor 0.10439371
lung cavity 0.1010775
atelectasis 0.08027959
pneumothorax -0.014359802
pleural effusion -0.029280338
tuberculosis -0.30615488


In [227]:
query = 'opacity in the left upper lung'
query_embedding = embedding_extractor.compute_text_embeddings([query], update_cache_on_disk=False)[0]
ref = 'lung opacity'
ref_embedding = embedding_extractor.compute_text_embeddings([ref], update_cache_on_disk=False)[0]

np.dot(query_embedding, ref_embedding)

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15196.75it/s]


0.30970702

In [258]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli" \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--background_findings_and_impression_json_filepath \
"/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json" \
--facts_similar_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=25,sf=129898,gf=44029,nf=5972,minsimth=0.650)(hash=686,1961964868541064712).pkl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157625400).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--mlp_batch_size 1000 \
--mlp_num_workers 4 \
--mlp_nli_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)" \
--mlp_nli_entailment_threshold 0.1723824292421341 \
--num_clusters 300 \
--max_negative_facts_per_report 500

Running _find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli()...
Reading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl...
n_reports: 227835
len(all_facts): 586352
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(all_facts): 769532
Loading /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=25,sf=129898,gf=44029,nf=5972,minsimth=0.650)(hash=686,1961964868541064712).pkl...
len(preserved_specific_facts): 129898
len(preserved_general_facts): 44029
len(preserved_negative_facts): 5972
len(anchor_facts): 25
len(all_facts): 772711
len(relevant_facts): 160797
Loading /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,15

In [260]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=160797,num_clusters=300,max_neg=500,ent_th=0.172)(hash=1231,1671346674768132252).pkl"

-rw-rw-r-- 1 pamessina socvis 900M Jan 17 11:21 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=160797,num_clusters=300,max_neg=500,ent_th=0.172)(hash=1231,1671346674768132252).pkl'


In [261]:
aux = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=160797,num_clusters=300,max_neg=500,ent_th=0.172)(hash=1231,1671346674768132252).pkl')

In [262]:
aux.keys()

dict_keys(['facts', 'embeddings', 'dicom_id_to_pos_facts', 'dicom_id_to_strong_neg_facts', 'dicom_id_to_weak_neg_facts', 'report_idxs_with_contradictions'])

In [263]:
len(aux['facts'])

772711

In [264]:
aux['embeddings'].shape

(772711, 128)

In [272]:
aux['report_idxs_with_contradictions'][0]

36

In [265]:
dicom_id_to_pos_facts = aux['dicom_id_to_pos_facts']
dicom_id_to_strong_neg_facts = aux['dicom_id_to_strong_neg_facts']
dicom_id_to_weak_neg_facts = aux['dicom_id_to_weak_neg_facts']

In [266]:
dicom_ids = list(dicom_id_to_pos_facts.keys())

In [267]:
integrated_report_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl')

In [277]:
from medvqa.datasets.mimiccxr import get_detailed_metadata_for_dicom_id, load_mimiccxr_reports_detailed_metadata
from medvqa.utils.files import read_txt
from medvqa.utils.logging import print_bold

In [278]:
mimiccxr_reports_detailed_metadata = load_mimiccxr_reports_detailed_metadata()

In [279]:
mimiccxr_reports_detailed_metadata['dicom_id_view_pos_pairs'][aux['report_idxs_with_contradictions'][0]]

[('59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7', 'AP'),
 ('e66f882f-e1aecd35-d9d369f5-65b2ae0a-371b82bb', 'LATERAL')]

In [281]:
aux['report_idxs_with_contradictions'][0]

36

In [275]:
type(image_views_dict)

dict

In [285]:
integrated_sentence_to_negative_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157625400).jsonl')

In [289]:
s2nf = { x['sentence']: x['ruled_out_abnormalities'] for x in integrated_sentence_to_negative_facts }

In [287]:
from medvqa.utils.files import load_json

In [288]:
bfis = load_json("/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json")

In [291]:
from nltk.tokenize import sent_tokenize

In [293]:
# dicom_id = dicom_ids[5040]
# dicom_id = '59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7'
dicom_id = dicom_ids[0]
metadata = get_detailed_metadata_for_dicom_id(dicom_id)[0]
ridx = metadata['report_index']
print(metadata)
print()
print()
print(read_txt(metadata['filepath']))
print()
print()
facts = integrated_report_facts[ridx]['facts']
print('. '.join(facts))
print()
print_bold('All facts:')
facts = integrated_report_facts[ridx]['facts']
print(facts)
print()
print_bold('Relevant positive facts:')
positive_facts = [aux['facts'][idx] for idx in dicom_id_to_pos_facts[dicom_id]]
print(positive_facts)
print()
print_bold('Excluded positive facts:')
print([f for f in facts if f not in positive_facts])
print()
print_bold('All strong negative facts:')
all_strong_negative_facts = set()
for x in (bfis[ridx]['findings'], bfis[ridx]['findings']):
    for s in sent_tokenize(x):
        nf = s2nf[s]
        if nf:
            print('------')
            print(s)
            print(nf)
            for k, v in nf.items():
                all_strong_negative_facts.update(v)
all_strong_negative_facts = list(all_strong_negative_facts)
print()
print_bold('Relevant strong negative facts:')
strong_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_strong_neg_facts[dicom_id]]
print(strong_negative_facts)
print()
print_bold('Excluded strong negative facts:')
excluded_strong_negative_facts = [x for x in all_strong_negative_facts if x not in strong_negative_facts]
print(excluded_strong_negative_facts)
print()
print_bold('Weak negative facts:')
weak_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_weak_neg_facts[dicom_id]]
for f in weak_negative_facts:
    print(f)

{'report_index': 0, 'part_id': 10, 'subject_id': 10703179, 'study_id': 58829627, 'dicom_id': '3f1d4d25-bc07dfbd-0d530479-35ef51a0-a45291cc', 'view_pos': 'PA', 'split': 'train', 'filepath': '/mnt/data/mimic-cxr/files/p10/p10703179/s58829627.txt', 'dicom_id_view_pos_pairs': [('3f1d4d25-bc07dfbd-0d530479-35ef51a0-a45291cc', 'PA'), ('768efca4-214691b8-11c6e785-87c7ef6a-691ddac9', 'LATERAL')]}


                                 FINAL REPORT
 EXAMINATION:  Chest radiograph
 
 INDICATION:  New dyspnea on exertion.
 
 TECHNIQUE:  Chest PA and lateral
 
 COMPARISON:  ___.
 
 FINDINGS: 
 
 Heart size is normal.  There are mild aortic knob calcifications. 
 Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. 
 Lungs are clear.  Pleural surfaces are clear without effusion or pneumothorax.
 
 IMPRESSION: 
 
 No acute cardiopulmonary abnormality.



normal heart size. mild aortic knob calcifications. unremarkable cardiomediastinal silhouette. unremarkable hilar contours. clea

In [679]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_relevant_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl"

Running _find_relevant_facts_with_respect_to_anchor_facts()...
len(anchor_facts): 25
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl...
len(unique_negative_facts): 21145
len(unique_negative_facts): 20889 (after removing banned facts)
Matching anchor fact 1/25: Aortic enlargement...
100%|████████████████████████████████| 652433/652433 [00:07<00:00, 84927.67it/s]
	Matched 2305 facts
Matching anchor fact 2/25: Atelectasis...
100%|████████████████████████████████| 652433/652433 [00:09<00:00, 70781.80it/s]
	Matched 32094 facts
Matching anchor fact 3/25: COPD...
100%|███████████████████████████████| 652433/652433 [00:06<00:00, 104968.53it/s]
	Matched 964 facts
Matching anchor fact 4/25: Calcifica

In [43]:
tmp = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=317035)(hash=411,354564789477999375).pkl')

In [44]:
tmp.keys()

dict_keys(['anchor_facts', 'relevant_facts', 'anchors_per_fact'])

In [45]:
tmp['anchor_facts']

['Aortic enlargement',
 'Atelectasis',
 'COPD',
 'Calcification',
 'Cardiomegaly',
 'Clavicle fracture',
 'Consolidation',
 'Edema',
 'Emphysema',
 'Enlarged PA',
 'ILD',
 'Infiltration',
 'Lung Opacity',
 'Lung cavity',
 'Lung cyst',
 'Lung tumor',
 'Mediastinal shift',
 'Nodule/Mass',
 'Pleural effusion',
 'Pleural thickening',
 'Pneumonia',
 'Pneumothorax',
 'Pulmonary fibrosis',
 'Rib fracture',
 'Tuberculosis']

In [684]:
from collections import Counter
Counter(len(x) for x in tmp['anchors_per_fact'])

Counter({2: 31916, 1: 283680, 3: 1417, 4: 22})

In [715]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli" \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--background_findings_and_impression_json_filepath \
"/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json" \
--facts_relevant_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=317035)(hash=411,354564789477999375).pkl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--mlp_batch_size 4000 \
--mlp_num_workers 8 \
--mlp_nli_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)" \
--mlp_nli_entailment_threshold 0.1723824292421341 \
--num_clusters 100 \
--max_negative_facts_per_report 500

Running _find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli()...
Reading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl...
n_reports: 227835
len(all_facts): 586352
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(all_facts): 769532
Loading /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=317035)(hash=411,354564789477999375).pkl...
len(relevant_facts): 317035
len(anchor_facts): 25
len(anchors_per_fact): 317035
len(all_facts): 773957
len(anchor_to_relevant_facts): 297
Min size: 1, Max size: 89199, Avg size: 1067.46
Loading /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl...
Average num

Saved kmeans labels to /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/kmeans_labels(34535504,3823435769704199565).pkl
Anchor (12, 17): 6424 facts, 71 non-empty clusters, min=1, max=1043, avg=90.48
Anchor (12,): 89199 facts, 99 non-empty clusters, min=1, max=5592, avg=901.00
Anchor (17,): 14984 facts, 85 non-empty clusters, min=1, max=3925, avg=176.28
Anchor (12, 17, 20): 35 facts, 8 non-empty clusters, min=1, max=13, avg=4.38
Anchor (3, 17): 764 facts, 29 non-empty clusters, min=1, max=331, avg=26.34
Anchor (3,): 9447 facts, 66 non-empty clusters, min=1, max=2046, avg=143.14
Anchor (21,): 15120 facts, 72 non-empty clusters, min=1, max=4774, avg=210.00
Anchor (3, 12): 385 facts, 27 non-empty clusters, min=1, max=197, avg=14.26
Anchor (1,): 27112 facts, 79 non-empty clusters, min=1, max=7748, avg=343.19
Anchor (18,): 38955 facts, 86 non-empty clusters, min=1, max=9941, avg=452.97
Anchor (13,): 1511 facts, 77 non-empty clusters, min=1, max=223, avg=19.62
Ancho

Elapsed time: 128.97s
100%|████████████████████████████████| 227835/227835 [00:11<00:00, 20123.77it/s]
Computing softmaxes over candidate weak negative facts...
metadata loaded from /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)/metadata.json
Creating model
EmbeddingBasedNLI
  embedding_dim: 128
  mlp_hidden_dims: [512, 256, 128]
  dropout: 0.0
Loading model weights from /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)
checkpoint_names = ['checkpoint_55_nli_acc=0.7684.pt']
Loading model weights from /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)/checkpoint_55_nli_acc=0.7684.pt
Estimated output memory size: 2172.80 MB
Computing softmaxes...
100%|█████████████████████████████████████| 28480/28480 [17:04<00:00,

In [5]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=317035,num_clusters=100,max_neg=500,ent_th=0.172)(hash=1200,2067390724509563020).pkl"

-rw-rw-r-- 1 pamessina socvis 943M Jan 20 14:13 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=317035,num_clusters=100,max_neg=500,ent_th=0.172)(hash=1200,2067390724509563020).pkl'


In [10]:
aux = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=317035,num_clusters=100,max_neg=500,ent_th=0.172)(hash=1200,2067390724509563020).pkl')

In [11]:
aux.keys()

dict_keys(['facts', 'embeddings', 'dicom_id_to_pos_facts', 'dicom_id_to_strong_neg_facts', 'dicom_id_to_weak_neg_facts', 'report_idxs_with_contradictions'])

In [12]:
len(aux['facts'])

773957

In [13]:
aux['embeddings'].shape

(773957, 128)

In [14]:
dicom_id_to_pos_facts = aux['dicom_id_to_pos_facts']
dicom_id_to_strong_neg_facts = aux['dicom_id_to_strong_neg_facts']
dicom_id_to_weak_neg_facts = aux['dicom_id_to_weak_neg_facts']

In [15]:
dicom_ids = list(dicom_id_to_pos_facts.keys())

In [16]:
integrated_report_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl')

In [17]:
from medvqa.datasets.mimiccxr import get_detailed_metadata_for_dicom_id, load_mimiccxr_reports_detailed_metadata
from medvqa.utils.files import read_txt
from medvqa.utils.logging import print_bold

In [18]:
mimiccxr_reports_detailed_metadata = load_mimiccxr_reports_detailed_metadata()

In [19]:
!ls -lh "/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl"

-rw-rw-r-- 1 pamessina pamessina 152M Jan 20 09:06 '/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl'


In [20]:
integrated_sentence_to_negative_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl')

In [21]:
s2nf = { x['sentence']: x['ruled_out_abnormalities'] for x in integrated_sentence_to_negative_facts }

In [22]:
from medvqa.utils.files import load_json

In [23]:
bfis = load_json("/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json")

In [24]:
from nltk.tokenize import sent_tokenize

In [25]:
len(aux['report_idxs_with_contradictions'])

12298

In [61]:
i = -1
# dicom_id = dicom_ids[5040]
# dicom_id = '59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7'
# dicom_id = 'ba822370-001bf2af-fc43ecb3-24b0e214-2271d081'
# dicom_id = '59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7'
# dicom_id = 'd9e61d03-d293db0c-9f8cd9cf-426c278f-b386d578'
# dicom_id = 'db97e24f-9cdb542e-ad3e0aa5-2be107fb-cf3ec751'
dicom_id = mimiccxr_reports_detailed_metadata['dicom_id_view_pos_pairs'][aux['report_idxs_with_contradictions'][i]][0][0]
# dicom_id = dicom_ids[0]
print(dicom_id)
metadata = get_detailed_metadata_for_dicom_id(dicom_id)[0]
ridx = metadata['report_index']
print(metadata)
print()
print()
print(read_txt(metadata['filepath']))
print()
print()
facts = integrated_report_facts[ridx]['facts']
print('. '.join(facts))
print()
print_bold('All facts:')
facts = integrated_report_facts[ridx]['facts']
print(facts)
print()
print_bold('Relevant positive facts:')
positive_facts = [aux['facts'][idx] for idx in dicom_id_to_pos_facts[dicom_id]]
print(positive_facts)
print()
print_bold('Excluded positive facts:')
print([f for f in facts if f not in positive_facts])
print()
print_bold('All strong negative facts:')
all_strong_negative_facts = set()
for x in (bfis[ridx]['findings'], bfis[ridx]['impression']):
    for s in sent_tokenize(x):
        nf = s2nf[s]
        print('------')
        print(s)
        print(nf)
        for k, v in nf.items():
            all_strong_negative_facts.update(v)
all_strong_negative_facts = list(all_strong_negative_facts)
print()
print_bold('Relevant strong negative facts:')
strong_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_strong_neg_facts[dicom_id]]
print(strong_negative_facts)
print()
print_bold('Excluded strong negative facts:')
excluded_strong_negative_facts = [x for x in all_strong_negative_facts if x not in strong_negative_facts]
print(excluded_strong_negative_facts)
print()
print_bold('Weak negative facts:')
weak_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_weak_neg_facts[dicom_id]]
for f in weak_negative_facts:
    print(f)

1a44a21d-a6c9aaf5-f606d7b4-6931086c-615f2b3d
{'report_index': 227822, 'part_id': 19, 'subject_id': 19535031, 'study_id': 59051226, 'dicom_id': '1a44a21d-a6c9aaf5-f606d7b4-6931086c-615f2b3d', 'view_pos': 'LATERAL', 'split': 'train', 'filepath': '/mnt/data/mimic-cxr/files/p19/p19535031/s59051226.txt', 'dicom_id_view_pos_pairs': [('1a44a21d-a6c9aaf5-f606d7b4-6931086c-615f2b3d', 'LATERAL'), ('95e18146-c058f3e2-02ce7450-f3f0279f-f8d71e27', 'AP')]}


                                 FINAL REPORT
 INDICATION:  Fever, here to evaluate for pneumonia.
 
 COMPARISON:  Chest radiograph dated ___.
 
 TECHNIQUE:  Upright AP and lateral radiographs of the chest.
 
 FINDINGS:  A left Port-A-Cath is unchanged in position with the tip
 terminating in the proximal right atrium.  The lungs are symmetrically well
 expanded and well aerated without focal consolidation, concerning for
 pneumonia, pleural effusion or pneumothorax.  The pulmonary vasculature is not
 engorged.  The cardiomediastinal and hilar c

In [58]:
q = 'region of consolidation not present prior to the exacerbation of pneumothorax'
for x in integrated_fact_metadata:
    if x['fact'] == q:
        print(x)
        break

{'fact': 'region of consolidation not present prior to the exacerbation of pneumothorax', 'metadata': {'reason': "The statement indicates that there was no region of consolidation observed before the exacerbation of pneumothorax, which is a relevant medical observation. It suggests a comparison of findings over time related to a specific condition. This information is pertinent to understanding the patient's respiratory status.", 'too_noisy_or_irrelevant': 'no', 'visually_observable': 'yes', 'category': 'disease', 'abnormality_status': 'completely normal or healthy', 'anatomical_location': 'does not apply', 'general_observation': 'no region of consolidation present prior to pneumothorax exacerbation'}, 'extraction_method': 'gpt-4o-mini'}


In [26]:
q = 'On the right, the hemidiaphragm now sharply seen with mild blunting of the costophrenic angle, consistent with decrease in the pleural effusion on this side.'
for x in integrated_sentence_to_negative_facts:
    if x['sentence'] == q:
        print(x)
        break

{'sentence': 'On the right, the hemidiaphragm now sharply seen with mild blunting of the costophrenic angle, consistent with decrease in the pleural effusion on this side.', 'ruled_out_abnormalities': {'sharply seen right hemidiaphragm': ['right hemidiaphragm elevation', 'significant right pleural effusion'], 'mild blunting of the costophrenic angle': ['large right pleural effusion', 'right costophrenic angle blunting']}, 'extraction_method': 'gpt-4o-mini'}


In [41]:
idxs = [i for i, x in enumerate(integrated_fact_metadata) if (x['metadata']['visually_observable'] == 'no' and
                                                              x['metadata']['abnormality_status'] in ('major abnormality', 'partial or minor abnormality'))]
len(idxs)

3350

In [56]:
integrated_fact_metadata[idxs[11]]

{'fact': 'mild peripharyngeal soft tissue thickening not visible on the current image',
 'metadata': {'reason': 'The statement describes a finding of mild peripharyngeal soft tissue thickening, which is a medical observation. However, it notes that this finding is not visible on the current image, indicating that it cannot be visually observed. This suggests that the observation is based on prior images or clinical history rather than the current X-ray.',
  'too_noisy_or_irrelevant': 'no',
  'visually_observable': 'no',
  'category': 'anatomical finding',
  'abnormality_status': 'partial or minor abnormality',
  'anatomical_location': 'peripharyngeal region',
  'general_observation': 'mild soft tissue thickening'},
 'extraction_method': 'gpt-4o-mini'}

In [58]:
filtered = [x['fact'] for x in integrated_fact_metadata if 'not visible' in x['fact']]
len(filtered)

191

In [60]:
filtered[1]

'cause of stridor not visible on conventional chest radiographs'

In [39]:
integrated_fact_metadata[idxs[11]]

{'fact': 'suggesting that the finding on the prior study at 00:13 represented dilated venous structures',
 'metadata': {'reason': 'The statement refers to findings from a prior study, indicating the presence of dilated venous structures. This suggests an anatomical observation that can be visually assessed on imaging. However, it does not provide a clear current observation from the chest X-ray itself.',
  'too_noisy_or_irrelevant': 'no',
  'visually_observable': 'no',
  'category': 'anatomical finding',
  'abnormality_status': 'does not apply',
  'anatomical_location': 'does not apply',
  'general_observation': 'dilated venous structures'},
 'extraction_method': 'gpt-4o-mini'}

In [35]:
integrated_sentence_facts = load_jsonl("/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_facts(59349684,50956825).jsonl")

In [46]:
q = 'Small regions of peribronchial opacification in the axillary region of the right upper lobe than right lung base could be pneumonia, perhaps due to aspiration, but diagnosis depends upon depends on clinical circumstances.'
for x in integrated_sentence_facts:
    if x['sentence'] == q:
        print(x)
        break

{'sentence': 'Small regions of peribronchial opacification in the axillary region of the right upper lobe than right lung base could be pneumonia, perhaps due to aspiration, but diagnosis depends upon depends on clinical circumstances.', 'facts': ['peribronchial opacification in the axillary region of the right upper lobe', 'peribronchial opacification in the axillary region of the right lung base', 'pneumonia in the axillary region of the right upper lobe', 'pneumonia in the axillary region of the right lung base', 'pneumonia due to aspiration in the axillary region of the right upper lobe', 'pneumonia due to aspiration in the axillary region of the right lung base', 'diagnosis depends on clinical circumstances'], 'extraction_method': 'gpt-3.5-turbo-0613'}


In [62]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_relevant_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(719172,256085081).jsonl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl"

Running _find_relevant_facts_with_respect_to_anchor_facts()...
len(anchor_facts): 25
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(719172,256085081).jsonl...
len(banned_facts): 123046
Loaded 596126 unique facts (skipped 123046 facts)
Loaded 305677 unique general facts
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl...
len(unique_negative_facts): 21145
len(unique_negative_facts): 20895 (after removing banned facts)
Matching anchor fact 1/25: Aortic enlargement...
100%|████████████████████████████████| 801118/801118 [00:09<00:00, 86685.13it/s]
	Matched 2754 facts
Matching anchor fact 2/25: Atelectasis...
100%|████████████████████████████████| 801118/801118 [00:09<00:00, 82016.16it/s]
	Matched 40553 facts
Matching anchor fact 3/25: COPD...
100%|███████████████████████████████| 801118/801118 [00:07<00:00, 107964.95it/s]
	Matched 1844 facts
Matching anchor fact 4/25: Calcific

In [73]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli" \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(77067916).jsonl" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(719172,256085081).jsonl" \
--background_findings_and_impression_json_filepath \
"/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json" \
--facts_relevant_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=404064)(hash=411,1888951042299108423).pkl" \
--integrated_sentence_to_negative_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--skip_nli \
--num_clusters 100 \
--max_negative_facts_per_report 500

Running _find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli()...
Reading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(77067916).jsonl...
n_reports: 227835
len(all_facts): 586223
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(719172,256085081).jsonl...
len(all_facts): 932516
Loading /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=404064)(hash=411,1888951042299108423).pkl...
len(relevant_facts): 404064
len(anchor_facts): 25
len(anchors_per_fact): 404064
len(all_facts): 936863
len(anchor_to_relevant_facts): 321
Min size: 1, Max size: 108978, Avg size: 1258.77
Loading /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl...
Average n

Elapsed time: 130.53s
100%|████████████████████████████████| 227835/227835 [00:11<00:00, 20123.35it/s]
Saving /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=404064,num_clusters=100,max_neg=500,skip_nli)(hash=980,2800495312613355816).pkl...
Done!


In [74]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=404064,num_clusters=100,max_neg=500,skip_nli)(hash=980,2800495312613355816).pkl"

-rw-rw-r-- 1 pamessina socvis 1.1G Jan 30 08:58 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=404064,num_clusters=100,max_neg=500,skip_nli)(hash=980,2800495312613355816).pkl'


In [75]:
aux = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=404064,num_clusters=100,max_neg=500,skip_nli)(hash=980,2800495312613355816).pkl')

In [76]:
aux.keys()

dict_keys(['facts', 'embeddings', 'dicom_id_to_pos_facts', 'dicom_id_to_strong_neg_facts', 'dicom_id_to_weak_neg_facts', 'report_idxs_with_contradictions'])

In [77]:
len(aux['facts'])

936863

In [78]:
aux['embeddings'].shape

(936863, 128)

In [79]:
dicom_id_to_pos_facts = aux['dicom_id_to_pos_facts']
dicom_id_to_strong_neg_facts = aux['dicom_id_to_strong_neg_facts']
dicom_id_to_weak_neg_facts = aux['dicom_id_to_weak_neg_facts']

In [80]:
dicom_ids = list(dicom_id_to_pos_facts.keys())
len(dicom_ids)

377110

In [81]:
integrated_report_facts = load_jsonl("/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(77067916).jsonl")

In [82]:
from medvqa.datasets.mimiccxr import get_detailed_metadata_for_dicom_id, load_mimiccxr_reports_detailed_metadata
from medvqa.utils.files import read_txt
from medvqa.utils.logging import print_bold

In [83]:
mimiccxr_reports_detailed_metadata = load_mimiccxr_reports_detailed_metadata()

In [84]:
!ls -lh "/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl"

-rw-rw-r-- 1 pamessina pamessina 152M Jan 20 09:06 '/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl'


In [85]:
integrated_sentence_to_negative_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_sentence_to_negative_facts(877989,157547664).jsonl')

In [86]:
s2nf = { x['sentence']: x['ruled_out_abnormalities'] for x in integrated_sentence_to_negative_facts }

In [87]:
from medvqa.utils.files import load_json

In [88]:
bfis = load_json("/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json")

In [89]:
from nltk.tokenize import sent_tokenize

In [90]:
len(aux['report_idxs_with_contradictions'])

32322

In [92]:
i = 0
# dicom_id = dicom_ids[5040]
# dicom_id = '59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7'
# dicom_id = 'ba822370-001bf2af-fc43ecb3-24b0e214-2271d081'
# dicom_id = '59438c05-50a25def-e33ef9fd-433adade-bdeaa4f7'
# dicom_id = 'd9e61d03-d293db0c-9f8cd9cf-426c278f-b386d578'
# dicom_id = 'db97e24f-9cdb542e-ad3e0aa5-2be107fb-cf3ec751'
dicom_id = mimiccxr_reports_detailed_metadata['dicom_id_view_pos_pairs'][aux['report_idxs_with_contradictions'][i]][0][0]
# dicom_id = dicom_ids[0]
print(dicom_id)
metadata = get_detailed_metadata_for_dicom_id(dicom_id)[0]
ridx = metadata['report_index']
print(metadata)
print()
print()
print(read_txt(metadata['filepath']))
print()
print()
facts = integrated_report_facts[ridx]['facts']
print('. '.join(facts))
print()
print_bold('All facts:')
facts = integrated_report_facts[ridx]['facts']
print(facts)
print()
print_bold('Relevant positive facts:')
positive_facts = [aux['facts'][idx] for idx in dicom_id_to_pos_facts[dicom_id]]
print(positive_facts)
print()
print_bold('Excluded positive facts:')
print([f for f in facts if f not in positive_facts])
print()
print_bold('All strong negative facts:')
all_strong_negative_facts = set()
for x in (bfis[ridx]['findings'], bfis[ridx]['impression']):
    for s in sent_tokenize(x):
        nf = s2nf[s]
        print('------')
        print(s)
        print(nf)
        for k, v in nf.items():
            all_strong_negative_facts.update(v)
all_strong_negative_facts = list(all_strong_negative_facts)
print()
print_bold('Relevant strong negative facts:')
strong_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_strong_neg_facts[dicom_id]]
print(strong_negative_facts)
print()
print_bold('Excluded strong negative facts:')
excluded_strong_negative_facts = [x for x in all_strong_negative_facts if x not in strong_negative_facts]
print(excluded_strong_negative_facts)
print()
print_bold('Weak negative facts:')
weak_negative_facts = [aux['facts'][idx] for idx in dicom_id_to_weak_neg_facts[dicom_id]]
for f in weak_negative_facts:
    print(f)

2d36ce8f-a684406b-92ed7650-4e6fcfb0-81c0e286
{'report_index': 28, 'part_id': 10, 'subject_id': 10306412, 'study_id': 58957659, 'dicom_id': '2d36ce8f-a684406b-92ed7650-4e6fcfb0-81c0e286', 'view_pos': 'LL', 'split': 'train', 'filepath': '/mnt/data/mimic-cxr/files/p10/p10306412/s58957659.txt', 'dicom_id_view_pos_pairs': [('2d36ce8f-a684406b-92ed7650-4e6fcfb0-81c0e286', 'LL'), ('6c027083-3e33f1ab-fbf4d513-48824ba8-09456e55', 'LL'), ('80841f89-3357d3b4-09b38b5b-d9351fa1-863aee93', nan)]}


                                 FINAL REPORT
 PA AND LATERAL CHEST X-RAY
 
 INDICATION:  Patient with HCC, TACE, hypoxia, volume overload, aspiration?
 interval change.
 
 COMPARISON:  ___.
 
 FINDINGS:
 
 Mild-to-moderate pulmonary edema has resolved.  There is persistent but
 improved bilateral streaky atelectatic bands.  There is no new lung
 consolidation.  No pleural effusion or pneumothorax.  Left-sided PICC line
 ends in mid SVC.
 
 CONCLUSION:
 
 1.  Resolving pulmonary edema.
 2.  Persistent but